In [1]:
!pip install rdflib
!pip install urllib3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 9.8 MB/s 
     |████████████████████████████████| 41 kB 140 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Data obtaining scripts
In below section I implemented list of methods that are used for get data from Internet

In [3]:
import urllib3
import json

urllib3.disable_warnings()

http = urllib3.PoolManager()
application_id = 'a12842cff8d85bdbdb9e779cca01397b'

# Script for obtain all 10 tier tanks info
def get_all_tanks_json():
  r = http.request('GET', f"https://api.tanki.su/wot/encyclopedia/vehicles/?application_id={application_id}&tier=10&fields=is_wheeled%2Ctag%2Ctank_id%2Ctype%2Cdescription%2Cshort_name%2Cnation%2Ctier%2Cname%2Cdefault_profile.hp%2Cdefault_profile.speed_forward%2Cdefault_profile.speed_backward%2Cdefault_profile.engine%2Cdefault_profile.ammo%2Cdefault_profile.modules%2Cdefault_profile.gun%2Cdefault_profile.turret%2Cdefault_profile.armor%2Cdefault_profile.rapid%2Cdefault_profile.siege")
  data = json.loads(r.data)
  res = data['data']
  return res

# Script for obtain player info
def get_user_info_by_nickname(nickname):
  ## get id by nickname
  r = http.request('GET', f"https://api.tanki.su/wot/account/list/?application_id={application_id}&search={nickname}")
  response = json.loads(r.data)
  data = response['data']
  player_id = None
  for player in data:
    if player['nickname'] == nickname:
      player_id = player['account_id']
  
  ## get info by id
  player_info = None
  if player_id != None:
    r = http.request('GET', f"https://api.tanki.su/wot/account/info/?application_id={application_id}&account_id={player_id}&fields=account_id%2Cglobal_rating%2Cclan_id%2Cstatistics.all%2Cstatistics.trees_cut%2Cnickname")
    data = json.loads(r.data)
    player_info = data['data'][f'{player_id}']

  return player_info

# Script for obtain stats on each tank by player
def get_player_stats_on_tank_by_id(player_id):
  r = http.request('GET', f"https://api.tanki.su/wot/tanks/stats/?account_id={player_id}&application_id={application_id}&in_garage=1&tank_id=20225%2C3473%2C7169%2C6193%2C15425%2C6145%2C5281%2C13569%2C32001%2C8705%2C2417%2C15953%2C14609%2C16897%2C16913%2C13857%2C3649%2C50849%2C6209%2C34049%2C7249%2C14113%2C9233%2C9297%2C52081%2C5265%2C5425%2C61697%2C4145%2C13825%2C20993%2C14881%2C13905%2C62513%2C4737%2C2721%2C22017%2C17217%2C55841%2C16961%2C58369%2C19489%2C6225%2C21537%2C2977%2C21761%2C8497%2C17729%2C5505%2C12369%2C50689%2C15905%2C13889%2C3681%2C63537%2C12049%2C57937%2C60977%2C6929%2C62257%2C35361%2C19217%2C3937%2C22273%2C11841%2C19473%2C10785%2C9489%2C12305%2C58961%2C8481%2C15697%2C2433%2C2929%2C19985%2C58641%2C5937%2C19969%2C46849%2C19201%2C15617%2C13089%2C19009&fields=account_id%2Cmax_xp%2Call%2Cmax_frags%2Cfrags%2Cmark_of_mastery%2Ctank_id")
  data = json.loads(r.data)
  res = data['data'][f'{player_id}']
  return res

# Script for obtain clan stats
def get_clan_info_by_tag(clan_tag):
  ## get id by tag
  r = http.request('GET', f"https://api.tanki.su/wot/clans/list/?application_id={application_id}&search={clan_tag}")
  response = json.loads(r.data)
  data = response['data']
  clan_id = None
  for clan in data:
    if clan['tag'] == clan_tag:
      clan_id = clan['clan_id']
  
  ## get info by id
  clan_info = None
  if clan_id != None:
    r = http.request('GET', f"https://api.tanki.su/wot/clans/info/?application_id={application_id}&clan_id={clan_id}&fields=members_count%2Cdescription%2Ccreator_name%2Cclan_id%2Cleader_name%2Ccreator_id%2Ctag%2Cmembers.account_id%2Cmembers.account_name%2Cold_name%2Cleader_id%2Cmotto%2Cold_tag%2Cname")
    data = json.loads(r.data)
    clan_info = data['data'][f'{clan_id}']

  return clan_info

# Script to obtain battles info
def get_battles_info():
  ## Opening JSON file
  f = open('gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/battles-results.json')
  
  ## JSON object as a dictionary
  battles = json.load(f)

  return battles

In [4]:
all_tanks = get_all_tanks_json()
print(f"number of tanks: {len(all_tanks)}")

player_json = get_user_info_by_nickname('preachooda')
print(f"{player_json['nickname']}'s id: {player_json['account_id']}")

player_stats_on_tanks_json = get_player_stats_on_tank_by_id(player_json['account_id'])
print(f"number of tanks that owned by {player_json['nickname']}: {len(player_stats_on_tanks_json)}")

clan_json = get_clan_info_by_tag('NE6P0')
print(f"number of {clan_json['name']} clan members: {len(clan_json['members'])}")

battles_results_json = get_battles_info()
print(f"number of battles: {len(battles_results_json)}")

number of tanks: 83
preachooda's id: 8438923
number of tanks that owned by preachooda: 31
number of Ne6p0 clan members: 90
number of battles: 7


In [5]:
tank = None
for id, info in all_tanks.items():
  tank = info
  break
  # if info['nation'] == 'czech':
  #   print(f"tank id = {id}; tank name = {info['short_name']}")

In [6]:
# Show tanks info
all_tanks

{'20225': {'description': 'Объект 730 вариант 4 представляет собой один из проектов САУ на базе танка Т-10 (Объект 730), подготовленный к 18 декабря 1952 г. При этом на предэскизном проекте указано, что это Объект 268. Проект предусматривал размещение орудия в неподвижной закрытой рубке, размещённой в кормовой части корпуса. Работы были свёрнуты на стадии проектирования. Причиной этого стала необходимость очень серьёзной переделки корпуса Объекта 730.',
  'short_name': 'Об. 268/4',
  'is_wheeled': False,
  'nation': 'ussr',
  'tag': 'R149_Object_268_4',
  'default_profile': {'engine': {'name': 'В-12-5В',
    'power': 1350,
    'weight': 1024,
    'tag': 'V_12_5F',
    'fire_chance': 0.12,
    'tier': 10},
   'rapid': None,
   'siege': None,
   'hp': 2000,
   'modules': {'gun_id': 44548,
    'suspension_id': 56834,
    'turret_id': None,
    'radio_id': 2823,
    'engine_id': 35845},
   'gun': {'move_down_arc': 5,
    'caliber': 152,
    'name': '152 мм М-53С',
    'weight': 4370,
    '

In [7]:
# Show common player stats
# player_json

In [8]:
# Show one entity of stats on concrete tank for player
# player_stats_on_tanks_json[0]

In [9]:
# Show clan info
# clan_json

In [10]:
# Show battles info
# battles_results_json

# Filling of ontology

In [13]:
from rdflib import Graph, URIRef, BNode, Literal

ontology_url = 'gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/tanks.owl'

# Create a Graph
tanks_ontology_graph = Graph()

# Parse in an RDF file
tanks_ontology_graph.parse(ontology_url)

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in tanks_ontology_graph:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in tanks_ontology_graph:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"tanks_ontology_graph has {len(tanks_ontology_graph)} statements.")

# print(tanks_ontology_graph.serialize(format="turtle"))

tanks_ontology_graph has 1023 statements.


In [14]:
#Class
Association_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Association')
Battle_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Battle')
BattleResults_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#BattleResults')
Clan_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Clan')
ClanMember_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#ClanMember')
Member_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Member')
Nation_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Nation')
Platoon_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Platoon')
PlatoonMember_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#PlatoonMember')
Player_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Player')
PlayerStatistic_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#PlayerStatistic')
PlayerStatisticOnConcreteTank_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#PlayerStatisticOnConcreteTank')
RapidMode_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#RapidMode')
SiegeMode_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#SiegeMode')
Tank_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Tank')
TankType_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#TankType')
Team_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#Team')
TeamMember_class = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#TeamMember')

#ObjectProperty
hasBattleResults_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasBattleResults')
hasClanMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasClanMember')
hasClanmate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasClanmate')
hasMate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasMate')
hasMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasMember')
hasNation_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasNation')
hasPlatoonMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasPlatoonMember')
hasPlatoonmate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasPlatoonmate')
hasRapidMode_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasRapidMode')
hasResults_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasResults')
hasSiegeMode_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasSiegeMode')
hasStatistic_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasStatistic')
hasTeam_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasTeam')
hasTeamMember_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasTeamMember')
hasTeammate_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasTeammate')
hasType_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hasType')
isMemberOf_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOf')
isMemberOfClan_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOfClan')
isMemberOfPlatoon_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOfPlatoon')
isMemberOfTeam_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isMemberOfTeam')
isResultsOfBattle_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isResultsOfBattle')
isResultsOfPlayer_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isResultsOfPlayer')
isStatisticOf_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#isStatisticOf')
playedOn_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#playedOn')
takePartIn_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#takePartIn')
tankStatisticOf_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tankStatisticOf')
tankStatisticOn_op = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tankStatisticOn')

#DatatypeProperty
accountId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#accountId')
avgDamageAssisted_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageAssisted')
avgDamageAssistedRadio_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageAssistedRadio')
avgDamageAssistedTrack_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageAssistedTrack')
avgDamageBlocked_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#avgDamageBlocked')
batResDamage_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResDamage')
batResFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResFrags')
batResPlayerClanTag_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResPlayerClanTag')
batResPlayerName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResPlayerName')
batResTank_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResTank')
batResXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResXp')
battleAvgXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleAvgXp')
battleId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleId')
battleResultsProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleResultsProperty')
battles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battles')
battlesOnStunningVehicles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battlesOnStunningVehicles')
capturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#capturePoints')
clanId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#clanId')
clanProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#clanProperty')
concreteTankAccountId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankAccountId')
concreteTankAvgDamageBlocked_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankAvgDamageBlocked')
concreteTankBattleAvgXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankBattleAvgXp')
concreteTankBattles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankBattles')
concreteTankBattlesOnStunningVehicles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankBattlesOnStunningVehicles')
concreteTankCapturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankCapturePoints')
concreteTankDamageDealt_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDamageDealt')
concreteTankDamageReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDamageReceived')
concreteTankDirectHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDirectHitsReceived')
concreteTankDraws_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDraws')
concreteTankDroppedCapturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankDroppedCapturePoints')
concreteTankExplosionHits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankExplosionHits')
concreteTankExplosionHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankExplosionHitsReceived')
concreteTankFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankFrags')
concreteTankHits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankHits')
concreteTankHitsPercents_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankHitsPercents')
concreteTankLosses_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankLosses')
concreteTankMarkOfMastery_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankMarkOfMastery')
concreteTankMaxFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankMaxFrags')
concreteTankMaxXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankMaxXp')
concreteTankNoDamageDirectHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankNoDamageDirectHitsReceived')
concreteTankPiercings_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankPiercings')
concreteTankPiercingsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankPiercingsReceived')
concreteTankShots_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankShots')
concreteTankSpotted_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankSpotted')
concreteTankStatisticProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankStatisticProperties')
concreteTankStunAssistedDamage_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankStunAssistedDamage')
concreteTankStunNumber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankStunNumber')
concreteTankSurvivedBattles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankSurvivedBattles')
concreteTankTankId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankTankId')
concreteTankTankingFactor_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankTankingFactor')
concreteTankWins_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankWins')
concreteTankXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#concreteTankXp')
creatorId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#creatorId')
creatorName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#creatorName')
damageDealt_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#damageDealt')
damageReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#damageReceived')
description_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#description')
directHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#directHitsReceived')
draws_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#draws')
droppedCapturePoints_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#droppedCapturePoints')
explosionHits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#explosionHits')
explosionHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#explosionHitsReceived')
frags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#frags')
globalRating_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#globalRating')
hits_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hits')
hitsPercents_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#hitsPercents')
leaderId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#leaderId')
leaderName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#leaderName')
losses_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#losses')
maxDamage_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#maxDamage')
maxFrags_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#maxFrags')
maxXp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#maxXp')
membersCount_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#membersCount')
motto_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#motto')
name_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#name')
nickName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#nickName')
noDamageDirectHitsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#noDamageDirectHitsReceived')
oldName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#oldName')
piercings_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#piercings')
piercingsReceived_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#piercingsReceived')
playerProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#playerProperty')
playerStatisticProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#playerStatisticProperties')
rapidModeProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidModeProperties')
rapidSpeedBackward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSpeedBackward')
rapidSpeedForward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSpeedForward')
rapidSuspensionSteeringLockAngle_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSuspensionSteeringLockAngle')
rapidSwitchOffTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSwitchOffTime')
rapidSwitchOnTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#rapidSwitchOnTime')
shots_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#shots')
siegeDispersion_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeDispersion')
siegeModeProperties_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeModeProperties')
siegeMoveDownArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeMoveDownArc')
siegeMoveUpArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeMoveUpArc')
siegeReloadTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeReloadTime')
siegeSpeedBackward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSpeedBackward')
siegeSuspensionTraverseSpeed_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSuspensionTraverseSpeed')
siegeSwitchOffTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSwitchOffTime')
siegeSwitchOnTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#siegeSwitchOnTime')
spotted_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#spotted')
stunNumber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#stunNumber')
survivedBattles_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#survivedBattles')
tag_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tag')
tankingFactor_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#tankingFactor')
treesCut_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#treesCut')
vechicleArmorHullFront_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorHullFront')
vechicleArmorHullRear_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorHullRear')
vechicleArmorHullSides_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorHullSides')
vechicleArmorTurretFront_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorTurretFront')
vechicleArmorTurretRear_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorTurretRear')
vechicleArmorTurretSides_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleArmorTurretSides')
vechicleDescription_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleDescription')
vechicleFireChance_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleFireChance')
vechicleGunAimTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunAimTime')
vechicleGunCaliber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunCaliber')
vechicleGunDispersion_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunDispersion')
vechicleGunFireRate_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunFireRate')
vechicleGunMoveDownArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunMoveDownArc')
vechicleGunMoveUpArc_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunMoveUpArc')
vechicleGunReloadTime_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunReloadTime')
vechicleGunTraverseSpeed_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleGunTraverseSpeed')
vechicleHp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleHp')
vechicleIsWheeled_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleIsWheeled')
vechicleModulesEngineId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesEngineId')
vechicleModulesGunId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesGunId')
vechicleModulesRadioId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesRadioId')
vechicleModulesSuspensionId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesSuspensionId')
vechicleModulesTurretId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleModulesTurretId')
vechicleName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleName')
vechiclePower_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechiclePower')
vechicleProperty_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleProperty')
vechicleShortName_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleShortName')
vechicleSpeedBackward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleSpeedBackward')
vechicleSpeedForward_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleSpeedForward')
vechicleTag_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTag')
vechicleTankId_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTankId')
vechicleTier_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTier')
vechicleTurretTraverseSpeed_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleTurretTraverseSpeed')
vechicleViewRange_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#vechicleViewRange')
wins_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#wins')
xp_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#xp')
batResPlatoonNumber_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#batResPlatoonNumber')
battleStatus_dp = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#battleStatus')

#Individual
AT_SPG_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#AT-SPG')
SPG_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#SPG')
china_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#china')
czech_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#czech')
france_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#france')
germany_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#germany')
heavyTank_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#heavyTank')
italy_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#italy')
japan_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#japan')
lightTank_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#lightTank')
mediumTank_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#mediumTank')
poland_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#poland')
sweden_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#sweden')
uk_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#uk')
usa_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#usa')
ussr_ind = URIRef('http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#ussr')

In [ ]:
# Fill graph with data from Internet and from files
## Create tanks triples
# for id, info in all_tanks.items():


# Save the entire Graph in the RDF Turtle format
# tanks_ontology_graph.serialize('gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/result.owl', format='turtle')